<a href="https://colab.research.google.com/github/Vicks-2019/NLP/blob/master/Predicting_similar_question_using_Doc2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# Import required libraries
import pandas as pd
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score

In [2]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [80]:
df = pd.read_csv('train.csv')
df.head()

,id,qid1,qid2,question1,question2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


In [81]:

# Check for null values
df[df.isnull().any(axis=1)]

,id,qid1,qid2,question1,question2
105780,105780,174363,174364,How can I develop android app?,NaN
201841,201841,303951,174364,How can I create an Android app?,NaN
363362,363362,493340,493341,NaN,My Chinese name is Haichao Yu. What English na...


In [82]:
# Drop rows with null Values
df.drop(df[df.isnull().any(axis=1)].index,inplace=True)

In [83]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [84]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
#     text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

In [85]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [99]:
df['question1'] = df.question1.apply(lambda x: clean_text(str(x)))
df.head()

,id,qid1,qid2,question1,question2
0,0,1,2,what be the step by step guide to invest in sh...,what be the step by step guide to invest in sh...
1,1,3,4,what be the story of kohinoor kohinoor diamond,what would happen if the indian government ste...
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increase by hack thr...
3,3,7,8,why be i mentally very lonely how can i solve it,find the remainder when math2324math be divide...
4,4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water


In [100]:
df['question2'] = df.question2.apply(lambda x: clean_text(str(x)))
df.head()

,id,qid1,qid2,question1,question2
0,0,1,2,what be the step by step guide to invest in sh...,what be the step by step guide to invest in sh...
1,1,3,4,what be the story of kohinoor kohinoor diamond,what would happen if the indian government ste...
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increase by hack thr...
3,3,7,8,why be i mentally very lonely how can i solve it,find the remainder when math2324math be divide...
4,4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water


In [125]:
df.shape

(404287, 5)

In [116]:
labeled_questions = []

for i in range(len(questions1)):
    # Question strings need to be separated into words
    # Each question needs a unique label
    labeled_questions.append(LabeledSentence(questions1[i].split(), df[df.index == i].qid1))
    labeled_questions.append(LabeledSentence(questions2[i].split(), df[df.index == i].qid2))
    

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  

In [126]:
# Split questions for computing similarity and determining the lengths of the questions.
questions1_split = []
for question in questions1:
    questions1_split.append(question.split())
    
questions2_split = []
for question in questions2:
    questions2_split.append(question.split())

In [127]:
# Determine the length of questions to select more optimal parameters.
lengths = []
for i in range(len(questions1_split)):
    lengths.append(len(questions1_split[i]))
    lengths.append(len(questions2_split[i]))
lengths = pd.DataFrame(lengths, columns=["count"])

In [128]:
lengths['count'].describe()

count    20000.000000
mean        11.063650
std          6.039199
min          1.000000
25%          7.000000
50%         10.000000
75%         13.000000
max        228.000000
Name: count, dtype: float64

In [129]:
# Build the model
model = Doc2Vec(dm = 1, min_count=1, window=10, size=150, sample=1e-4, negative=10)
model.build_vocab(labeled_questions)

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [135]:
# Train the model with 50 epochs

for epoch in range(50):
    model.train(labeled_questions,epochs=model.iter,total_examples=model.corpus_count)
    print("Epoch #{} is complete.".format(epoch+1))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  after removing the cwd from sys.path.


Epoch #1 is complete.
Epoch #2 is complete.
Epoch #3 is complete.
Epoch #4 is complete.
Epoch #5 is complete.
Epoch #6 is complete.
Epoch #7 is complete.
Epoch #8 is complete.
Epoch #9 is complete.
Epoch #10 is complete.
Epoch #11 is complete.
Epoch #12 is complete.
Epoch #13 is complete.
Epoch #14 is complete.
Epoch #15 is complete.
Epoch #16 is complete.
Epoch #17 is complete.
Epoch #18 is complete.
Epoch #19 is complete.
Epoch #20 is complete.
Epoch #21 is complete.
Epoch #22 is complete.
Epoch #23 is complete.
Epoch #24 is complete.
Epoch #25 is complete.
Epoch #26 is complete.
Epoch #27 is complete.
Epoch #28 is complete.
Epoch #29 is complete.
Epoch #30 is complete.
Epoch #31 is complete.
Epoch #32 is complete.
Epoch #33 is complete.
Epoch #34 is complete.
Epoch #35 is complete.
Epoch #36 is complete.
Epoch #37 is complete.
Epoch #38 is complete.
Epoch #39 is complete.
Epoch #40 is complete.
Epoch #41 is complete.
Epoch #42 is complete.
Epoch #43 is complete.
Epoch #44 is complet

In [136]:
doc2vec_scores = []
for i in range(len(questions1_split)):
    # n_similarity computes the cosine similarity in Doc2Vec
    score = model.n_similarity(questions1_split[i],questions2_split[i])
    doc2vec_scores.append(score)
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  after removing the cwd from sys.path.


In [144]:
df1['similarity'] = doc2vec_scores

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [145]:
df1.head()

,id,qid1,qid2,question1,question2,similarity
0,0,1,2,what be the step by step guide to invest in sh...,what be the step by step guide to invest in sh...,0.962094
1,1,3,4,what be the story of kohinoor kohinoor diamond,what would happen if the indian government ste...,0.598492
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increase by hack thr...,0.701840
3,3,7,8,why be i mentally very lonely how can i solve it,find the remainder when math2324math be divide...,0.092661
4,4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,0.616106


In [146]:
score = model.n_similarity(questions1_split[i],questions2_split[i])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


In [147]:
score

0.76755005

In [148]:
model.most_similar('good')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('best', 0.4686870276927948),
 ('than', 0.3967460095882416),
 ('like', 0.3777703642845154),
 ('bad', 0.36278751492500305),
 ('their', 0.3612136244773865),
 ('better', 0.34966081380844116),
 ('students', 0.34804970026016235),
 ('easy', 0.30829674005508423),
 ('funniest', 0.3079981803894043),
 ('ethical', 0.30390504002571106)]